# Solution

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

# Set random seed for reproducibility
np.random.seed(42)

## Problem Analysis

<!-- Add your analysis here -->

How would you get the Confidence Interval of Median?

In [ ]:
import numpy as np

def bootstrap_ci_median(x, n_boot=10_000, ci=0.95, random_state=None):
    """
    Percentile bootstrap confidence interval for the median.
    x: 1D array-like of observations
    n_boot: number of bootstrap resamples
    ci: confidence level (e.g., 0.95)
    """
    rng = np.random.default_rng(random_state)
    x = np.asarray(x)
    n = len(x)

    # 1) Generate bootstrap medians
    boot_medians = np.empty(n_boot)
    for b in range(n_boot):
        sample_idx = rng.integers(0, n, size=n)   # sample with replacement
        boot_medians[b] = np.median(x[sample_idx])

    # 2) Percentile bounds
    alpha = 1 - ci
    lower = np.percentile(boot_medians, 100 * (alpha / 2))
    upper = np.percentile(boot_medians, 100 * (1 - alpha / 2))

    return {
        "median": float(np.median(x)),
        "ci_lower": float(lower),
        "ci_upper": float(upper),
        "level": ci
    }

# Example usage
data = [3, 5, 7, 9, 12, 15, 18, 22, 25]
result = bootstrap_ci_median(data, n_boot=10_000, ci=0.95, random_state=42)
print(result)
# {'median': ..., 'ci_lower': ..., 'ci_upper': ..., 'level': 0.95}

## Conclusion

<!-- Summarize findings and insights -->

When the problem shifts from “mean” to “median,” do not panic or reach for awkward normal approximations. A clean, implementation‑ready answer is to use bootstrap resampling and read off the percentile‑based confidence interval from the empirical distribution of the median.​

This is exactly the kind of thinking that separates textbook knowledge from practical, interview‑ready data science.